In [127]:
import pandas as pd
import oanda_utils
import plotly.graph_objects as go

In [128]:
def plot_candles(df_plot, df_markers=None):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c, line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))

    if df_markers is not None:
        fig.add_trace(go.Candlestick(
            x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c, line=dict(width=1), opacity=1,
            increasing_fillcolor='#3480eb',
            decreasing_fillcolor='#3480eb',
            increasing_line_color='#3480eb',
            decreasing_line_color='#3480eb'
        ))

    fig.update_layout(width=1000, height=400, margin=dict(l=10,r=10,b=10,t=10), font=dict(size=10, color='#e1e1e1'), paper_bgcolor='#1e1e1e', plot_bgcolor='#1e1e1e')
    fig.update_xaxes(
        gridcolor='#1F292F', showgrid=True, fixedrange=True, rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor='#1F292F', 
        showgrid=True
    )
    fig.show()

In [129]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(oanda_utils.get_his_data_filename(pair, granularity))

In [130]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [131]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,...,DIST_TOP_PERC,DIST_BOTTOM_PERC,PREV_BODY_RANGE,PREV_BODY_PERC,PREV_DIRECTION,HAMMER,SPINNING_TOP,DOJI,MARUBOZU,ENGULFING
3,2018-01-02 02:00:00+00:00,112.720,112.751,112.666,112.688,0.085,0.032,112.7085,0.376471,-1,...,0.258824,0.258824,0.057,0.270142,1.0,False,False,False,False,False
6,2018-01-02 06:00:00+00:00,112.691,112.700,112.249,112.252,0.451,0.439,112.4745,0.973392,-1,...,0.006652,0.006652,0.032,0.376471,-1.0,False,False,False,True,False
9,2018-01-02 10:00:00+00:00,112.254,112.282,112.086,112.124,0.196,0.130,112.1840,0.663265,-1,...,0.193878,0.193878,0.439,0.973392,-1.0,False,False,False,False,False
12,2018-01-02 14:00:00+00:00,112.122,112.304,112.058,112.285,0.246,0.163,112.1810,0.662602,1,...,0.077236,0.260163,0.130,0.663265,-1.0,False,False,False,False,True
15,2018-01-02 18:00:00+00:00,112.288,112.354,112.244,112.291,0.110,0.003,112.2990,0.027273,1,...,0.572727,0.400000,0.163,0.662602,1.0,False,False,True,False,False


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4655 entries, 3 to 1224
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   time              4655 non-null   datetime64[ns, tzutc()]
 1   mid_o             4655 non-null   float64                
 2   mid_h             4655 non-null   float64                
 3   mid_l             4655 non-null   float64                
 4   mid_c             4655 non-null   float64                
 5   RANGE             4655 non-null   float64                
 6   BODY_RANGE        4655 non-null   float64                
 7   CENTER            4655 non-null   float64                
 8   BODY_PERC         4655 non-null   float64                
 9   DIRECTION         4655 non-null   int64                  
 10  DIST_TOP          4655 non-null   float64                
 11  DIST_BOTTOM       4655 non-null   float64                
 12  DIST_T

In [133]:
SMALL_BODY = 0.2
CLOSE_DISTANCE = 0.15
DOJI_BODY = 0.05
FULL_BODY = 0.95
CENTER_DISTANCE_HIGH = 0.55
CENTER_DISTANCE_LOW = 0.45
BIG_BODY = 0.6
ENGULFING_FACTOR = 1.1


def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_c - row.mid_l

def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l

def apply_hammer(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CLOSE_DISTANCE or row.DIST_BOTTOM_PERC < CLOSE_DISTANCE:
            return True
    return False

def apply_spinning_top(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CENTER_DISTANCE_HIGH and row.DIST_TOP_PERC > CENTER_DISTANCE_LOW:
            return True
    return False

def apply_engulfing(row):
    if row.PREV_DIRECTION != row.DIRECTION:
        if row.PREV_BODY_PERC > BIG_BODY and row.BODY_PERC > BIG_BODY:
            if row.BODY_RANGE > row.PREV_BODY_RANGE * ENGULFING_FACTOR:
                return True
    return False

def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] = df.BODY_RANGE / df.RANGE
    df['DIRECTION'] = df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x>= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis=1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis=1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE

    df['PREV_BODY_RANGE'] = df.BODY_RANGE.shift(1)
    df['PREV_BODY_PERC'] = df.BODY_PERC.shift(1)
    df['PREV_DIRECTION'] = df.DIRECTION.shift(1)

    df.dropna(inplace=True)

    return df

def apply_patterns(df):
    df['HAMMER'] = df.apply(apply_hammer, axis=1)
    df['SPINNING_TOP'] = df.apply(apply_spinning_top, axis=1)
    df['DOJI'] = df['BODY_PERC'].apply(lambda x: True if x < DOJI_BODY else False)
    df['MARUBOZU'] = df['BODY_PERC'].apply(lambda x: True if x > FULL_BODY else False)
    df['ENGULFING'] = df.apply(apply_engulfing, axis=1)
    return df

The LAMBDA function is like declaring a mini insta-function. A similar function could be writen as:

def function_f(x):
    if x >= 0: 
        return 1
    else:
        return -1

In [134]:
df = df_raw[cols].copy()
df = apply_stats(df)
df = apply_patterns(df)
df.head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,...,DIST_TOP_PERC,DIST_BOTTOM_PERC,PREV_BODY_RANGE,PREV_BODY_PERC,PREV_DIRECTION,HAMMER,SPINNING_TOP,DOJI,MARUBOZU,ENGULFING
3,2018-01-02 02:00:00+00:00,112.720,112.751,112.666,112.688,0.085,0.032,112.7085,0.376471,-1,...,0.258824,0.258824,0.057,0.270142,1.0,False,False,False,False,False
6,2018-01-02 06:00:00+00:00,112.691,112.700,112.249,112.252,0.451,0.439,112.4745,0.973392,-1,...,0.006652,0.006652,0.032,0.376471,-1.0,False,False,False,True,False
9,2018-01-02 10:00:00+00:00,112.254,112.282,112.086,112.124,0.196,0.130,112.1840,0.663265,-1,...,0.193878,0.193878,0.439,0.973392,-1.0,False,False,False,False,False
12,2018-01-02 14:00:00+00:00,112.122,112.304,112.058,112.285,0.246,0.163,112.1810,0.662602,1,...,0.077236,0.260163,0.130,0.663265,-1.0,False,False,False,False,True
15,2018-01-02 18:00:00+00:00,112.288,112.354,112.244,112.291,0.110,0.003,112.2990,0.027273,1,...,0.572727,0.400000,0.163,0.662602,1.0,False,False,True,False,False


In [137]:
df_plot = df.iloc[700:800]
# df_markers = df_plot[(df_plot.MARUBOZU==True) | (df_plot.DOJI==True | (df_plot.HAMMER==True) | (df_plot.SPINNING_TOP==True)) |  (df_plot.ENGULFING==True)]
df_markers = df_plot[(df_plot.ENGULFING==True)]

In [138]:
plot_candles(df_plot, df_markers)